In [12]:
import re

In [89]:
import pandas as pd

In [36]:
punctuation = '''!#$%&*+, -./:;=?@\^_`|~'''

In [381]:
with open("Evendsl.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

In [382]:
dictlines = []
for i in range(len(lines)):
    if (i+1)%2 == 0:
        dictlines.append(lines[i].strip())

In [383]:
comments = ['междом.', 'тотем.', '(фолькл.)', 'фолькл.', 'обращ.', 'зап.', 'иносказ.',
            'собств.', 'образн.', 'устар.', 'изобр.', '(< рус.)', 'ласк.']
def get_word_meaning(sent):
 
    pattern = '(?P<word>(.+?))(\s(I*?)(,)*)*\s(\(.+?\))*(?P<sec_part>[^(].+)'
    result = re.search(pattern, sent)
    if result:  # получили первое слово и остальное
        meaning = result.group('sec_part')
        for comment in comments:
            meaning = meaning.replace(comment, '')
        meaning = meaning.strip(',')
        meaning = meaning.strip()
        base = result.group('word').strip(',')
        base = base.lower()
        pattern_transl = '(\([а-яё].+?\))*([0-9]\)\s)*(?P<transl>([а-яё]+))'
        result2 = re.search(pattern_transl, meaning)
        if result2:  # получили первое слово маленькими буквами из определения
            transl = result2.group('transl')
            return [result.group('word').strip(punctuation), result.group('sec_part'), base.strip(punctuation), transl]

    return sent

In [384]:
lines_data = []
for dictline in dictlines:
    lines_data.append(get_word_meaning(dictline))
df = pd.DataFrame(lines_data, columns = ['word', 'sec_part', 'base', 'transl'])

In [385]:
# Даём первеводы тому, где "см. другое_слово"
for index, row in df.iterrows():
    if row['transl'] == 'см':
        try:
            sinonim = row['sec_part'][4:].strip(punctuation)
            row['transl'] = df.loc[df.word == sinonim.upper()]['transl'].values[0]
        except Exception:
            sinonim = 0
            for word in row['sec_part'][4:].strip(punctuation).split(' '):
                if word.islower():
                    if word != 'см.':
                        sinonim = word.strip(punctuation)
                        break
            row['transl'] = df.loc[df.word == sinonim.upper()]['transl'].values[0]

In [386]:
# Добавляем 2е-4е слова заглавными буквами
for index, row in df.iterrows():
    pattern = '(?P<word>(.+?))\s(\(.+?\)\s)*(\(.+?\)\s)*(?P<sec_part>[^(].+)'
   
    result = re.search(pattern, row['sec_part'])
    if result:
        if result.group('word').isupper():
            df.loc[len(df.index)] = [result.group('word').strip(punctuation), row['sec_part'], result.group('word').strip(punctuation).lower(), row['transl']]
            
            result2 = re.search(pattern, result.group('sec_part'))
            if result2:
                if result2.group('word').strip(punctuation).isupper():
                    df.loc[len(df.index)] = [result2.group('word').strip(punctuation), row['sec_part'], result2.group('word').strip(punctuation).lower(), row['transl']]
                    
                    result3 = re.search(pattern, result2.group('sec_part'))
                    if result3:
                        if result3.group('word').isupper():
                            df.loc[len(df.index)] = [result3.group('word'), row['sec_part'], result3.group('word').lower(), row['transl']]

In [387]:
# Убираем часть морфологии
for index, row in df.iterrows():
    if row['base'].startswith('х'):  # убрали х в начале
        row['base'] = row['base'][1:]
    if row['base'].endswith('ай') or row['base'].endswith('эй'): # у глаголов забрали -purp-...
        if len(row['base']) > 3:
            row['base'] = row['base'][:-3]+'-'


In [389]:
df

,word,sec_part,base,transl
0,А̄-А,"междом., крик при легком испуге, удивлении. А...",а̄-а,крик
1,АБА,"обращ. зап. папа, отец (в языке детей); ср. ам...",аба,папа
2,АБА,АБАЛКАН так говорят дети про что-л. красивое.,аба,так
3,АБАГА,тотем. 1) дедушка (медведь); 2) зап. дед; ср. ...,абага,дедушка
4,АБАЛ,"мало, нехватка, дешево (о цене); ср. адыкун. ...",абал,мало
...,...,...,...,...
15053,Я̄НЮЧ,"Я̄НЮЧ (совм. п. от мест. я̄к) с кем, с чем.",я̄нюч,совм
15054,ЯРИМКАР,"ЯРИМКАР камни, образовавшиеся от разрушения скал.",яримкар,камни
15055,Я̄РУЧИН,"Я̄РУЧИН, Я̄РУЧИНМАЙ 1) проверка, осмотр, обсле...",я̄ручин,проверка
15056,Я̄РУЧИНМАЙ,"Я̄РУЧИН, Я̄РУЧИНМАЙ 1) проверка, осмотр, обсле...",я̄ручин-,проверка


In [390]:
df.to_csv('evendict_bases.csv', sep='±')